## Semantic similarity 

In this notebook I use all similarity metrics from the NLTK library.
Given a concept from the McRae as well as a list of the entire dataset concepts, this functions obtain the closest concepts to the input concept. 

### Header, getting list of concepts and applying a similarity metric.

In [30]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

def get_concepts_list ():
    "Returns a list of strings: the names of the concepts"
    df = pd.read_excel('../McRaedataset/CONCS_Synset_brm.xlsx')
    return map(str, list(df['Concept']))
    
def get_synset (concept):
    "Given a concept name (string) it returns its synset (string)"
    # Dataframe for excel document
    df = pd.read_excel('../McRaedataset/CONCS_Synset_brm.xlsx')
    row = df.loc[df['Concept'] == concept]
    return str(list(row['Synset'] )[0])

def Apply_sim_metric ( similarity_metric, num, in_concept, corpus = None):
    "Given a similarity_metric function it returns a list of the num closest concepts to 'concept'"
    dist_list = []
    for c in Concepts:
        c_synset = wn.synset( get_synset(c) )
        if corpus:
            dist_list.append([c, round(similarity_metric(in_concept, c_synset, corpus), 3) ])
        else:
            dist_list.append([c, round(similarity_metric(in_concept, c_synset), 3) ])
    return sorted(dist_list, key = lambda r : r[1], reverse = True ) [:num]

## Closest concepts to an input concept

In [31]:
# List of concepts
Concepts = get_concepts_list() 

#Input concept
concept = wn.synset( get_synset("airplane") )

### Path similarity

This metric is based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy.
The score is in the range 0 to 1. 

In [32]:
print Apply_sim_metric(wn.path_similarity, 20, concept )

[['airplane', 1.0], ['jet', 0.5], ['helicopter', 0.333], ['boat', 0.167], ['ship', 0.167], ['sled', 0.167], ['sleigh', 0.167], ['yacht', 0.167], ['bike', 0.143], ['missile', 0.143], ['sailboat', 0.143], ['scooter', 0.143], ['skateboard', 0.143], ['tank_(army)', 0.143], ['trailer', 0.143], ['tricycle', 0.143], ['unicycle', 0.143], ['wagon', 0.143], ['bus', 0.125], ['canoe', 0.125]]


### Leacock-Chodorow similarity

Return a score denoting how similar two word senses are, based on the shortest path that connects the senses (as above) and the maximum depth of the taxonomy in which the senses occur. 

The relationship is given as -log(p/2d) where p is the shortest path length and d the taxonomy depth.

In [33]:
print Apply_sim_metric(wn.lch_similarity, 20, concept )

[['airplane', 3.638], ['jet', 2.944], ['helicopter', 2.539], ['boat', 1.846], ['ship', 1.846], ['sled', 1.846], ['sleigh', 1.846], ['yacht', 1.846], ['bike', 1.692], ['missile', 1.692], ['sailboat', 1.692], ['scooter', 1.692], ['skateboard', 1.692], ['tank_(army)', 1.692], ['trailer', 1.692], ['tricycle', 1.692], ['unicycle', 1.692], ['wagon', 1.692], ['bus', 1.558], ['canoe', 1.558]]


### Wu-Palmer Similarity

Return a score denoting how similar two word senses are, based on the depth of the two senses in the taxonomy and that of their Least Common Subsumer (most specific ancestor node).

In [34]:
print Apply_sim_metric(wn.wup_similarity, 20, concept )

[['airplane', 1.0], ['jet', 0.96], ['helicopter', 0.917], ['boat', 0.783], ['ship', 0.783], ['yacht', 0.783], ['sled', 0.762], ['sleigh', 0.762], ['sailboat', 0.75], ['bike', 0.727], ['missile', 0.727], ['scooter', 0.727], ['skateboard', 0.727], ['tank_(army)', 0.727], ['tricycle', 0.727], ['unicycle', 0.727], ['wagon', 0.727], ['canoe', 0.72], ['trailer', 0.7], ['cart', 0.696]]


## Information content-based metrics

### *wordnet_ic* 

**Information Content:** Load an information content file from the wordnet_ic corpus (or can be a custom made inf cont dictionary)

Note that for any similarity measure that uses information content, the result is dependent on the corpus used to generate the information content and the specifics of how the information content was created.

In [35]:
## Information content...
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

### Resnik similarity.
Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node).

In [36]:
print Apply_sim_metric(wn.res_similarity, 20, concept, brown_ic)
print Apply_sim_metric(wn.res_similarity, 20, concept, semcor_ic)

[['airplane', 8.871], ['jet', 8.871], ['helicopter', 8.643], ['boat', 7.221], ['canoe', 7.221], ['sailboat', 7.221], ['ship', 7.221], ['submarine', 7.221], ['yacht', 7.221], ['ambulance', 5.922], ['bike', 5.922], ['buggy', 5.922], ['car', 5.922], ['cart', 5.922], ['dunebuggy', 5.922], ['jeep', 5.922], ['limousine', 5.922], ['missile', 5.922], ['motorcycle', 5.922], ['scooter', 5.922]]
[['airplane', 7.916], ['jet', 7.916], ['helicopter', 7.591], ['boat', 6.472], ['canoe', 6.472], ['sailboat', 6.472], ['ship', 6.472], ['submarine', 6.472], ['yacht', 6.472], ['ambulance', 5.531], ['bike', 5.531], ['buggy', 5.531], ['car', 5.531], ['cart', 5.531], ['dunebuggy', 5.531], ['jeep', 5.531], ['limousine', 5.531], ['missile', 5.531], ['motorcycle', 5.531], ['scooter', 5.531]]


### Jiang-Conrath similarity

Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node) and that of the two input Synsets. 
The relationship is given by the equation 1 / (IC(s1) + IC(s2) - 2 * IC(lcs)).

In [37]:
print Apply_sim_metric(wn.jcn_similarity, 20, concept, brown_ic)
print Apply_sim_metric(wn.jcn_similarity, 20, concept, semcor_ic)

[['airplane', 1e+300], ['jet', 0.477], ['ship', 0.363], ['boat', 0.323], ['submarine', 0.219], ['car', 0.217], ['helicopter', 0.202], ['truck', 0.178], ['canoe', 0.165], ['bike', 0.159], ['missile', 0.158], ['train', 0.148], ['van', 0.141], ['yacht', 0.141], ['wagon', 0.139], ['limousine', 0.137], ['sailboat', 0.133], ['bus', 0.132], ['sled', 0.128], ['sleigh', 0.128]]
[['airplane', 1e+300], ['ship', 0.402], ['jet', 0.363], ['boat', 0.3], ['car', 0.259], ['submarine', 0.209], ['wagon', 0.195], ['train', 0.189], ['truck', 0.185], ['missile', 0.182], ['tank_(army)', 0.15], ['canoe', 0.148], ['yacht', 0.148], ['cart', 0.146], ['jeep', 0.146], ['bike', 0.143], ['taxi', 0.143], ['ambulance', 0.138], ['trolley', 0.138], ['gun', 0.134]]


### Lin similarity
Return a score denoting how similar two word senses are, based on the Information Content (IC) of the Least Common Subsumer (most specific ancestor node) and that of the two input Synsets. 
The relationship is given by the equation 2 * IC(lcs) / (IC(s1) + IC(s2)).

In [38]:
print Apply_sim_metric(wn.lin_similarity, 20, concept, brown_ic)
print Apply_sim_metric(wn.lin_similarity, 20, concept, semcor_ic)

[['airplane', 1.0], ['jet', 0.894], ['ship', 0.84], ['boat', 0.823], ['helicopter', 0.777], ['submarine', 0.759], ['car', 0.719], ['canoe', 0.704], ['truck', 0.679], ['yacht', 0.67], ['sailboat', 0.658], ['bike', 0.654], ['missile', 0.652], ['train', 0.631], ['van', 0.625], ['wagon', 0.621], ['limousine', 0.618], ['bus', 0.603], ['sled', 0.603], ['sleigh', 0.603]]
[['airplane', 1.0], ['jet', 0.852], ['ship', 0.839], ['boat', 0.795], ['car', 0.742], ['submarine', 0.73], ['wagon', 0.684], ['train', 0.673], ['truck', 0.672], ['missile', 0.669], ['canoe', 0.658], ['yacht', 0.658], ['tank_(army)', 0.624], ['cart', 0.618], ['jeep', 0.618], ['bike', 0.612], ['taxi', 0.612], ['ambulance', 0.605], ['trolley', 0.605], ['tractor', 0.595]]
